In [1]:
import torch

In [7]:
words = open('../datasets/names.txt', 'r').read().splitlines()

In [12]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [44]:
xs, ys = [], []
for w in words:
    chs = ["."] + ["."] + list(w) + ["."]
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        xs.append((ix1, ix2))
        ys.append(ix3)

# xs = torch.tensor(xs)
# ys = torch.tensor(ys)


In [62]:
import torch.nn.functional as F

num_classes = 27**2  

xs_set = sorted(list(set(xs)))
xs_set = [(i, j) for i in range(27) for j in range(27)] 

mapping = {tup: idx for idx, tup in enumerate(xs_set)}
indices = torch.tensor([mapping[pair] for pair in xs])

one_hot = F.one_hot(indices, num_classes=num_classes).float()

In [65]:

one_hot


tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.]])